In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns

In [2]:
file_path = r"C:\Users\prana\Desktop\Data Science\NLP\ANN\BankCustomerChurnScaled.csv"
df = pd.read_csv(file_path)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,0.538,1,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0
1,0.516,1,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1
2,0.304,1,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0
3,0.698,1,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0
4,1.000,1,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0,0.283784,0.5,0.000000,0.333333,1,0,0.481341,0,1,0,0
9996,0.332,0,0.229730,1.0,0.228657,0.000000,1,1,0.508490,0,1,0,0
9997,0.718,1,0.243243,0.7,0.000000,0.000000,0,1,0.210390,1,1,0,0
9998,0.844,0,0.324324,0.3,0.299226,0.333333,1,0,0.464429,1,0,1,0


## Handling Imbalanced Datadet

### SMOTE - Synthetic Minority Over Sampling Technique

In [4]:
X = df.drop('Exited',axis=1)
y=df['Exited']

In [5]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [28]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = 'minority')

X,Y = smote.fit_resample(X,y)

y.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [11]:
X_train.shape,X_test.shape

((12740, 12), (3186, 12))

In [12]:
y_train.value_counts()

0    6370
1    6370
Name: Exited, dtype: int64

In [13]:
import tensorflow as tf
from tensorflow import keras

In [19]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

model = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.0001)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=100, callbacks=[early_stopping])


Epoch 1/100
399/399 [==============================] - 2s 2ms/step - loss: 0.6461 - accuracy: 0.6221
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5931 - accuracy: 0.6859
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5700 - accuracy: 0.7053
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5607 - accuracy: 0.7123
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7173
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5516 - accuracy: 0.7203
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7223
Epoch 8/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5459 - accuracy: 0.7205
Epoch 9/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5443 - accuracy: 0.7225
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5410 - accura

In [21]:
model.evaluate(X_test,y_test)

100/100 [==============================] - 0s 2ms/step - loss: 0.4477 - accuracy: 0.7884


[0.4476742744445801, 0.7884494662284851]

In [23]:
yp = model.predict(X_test)
yp[:5]

y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

100/100 [==============================] - 0s 2ms/step


In [24]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.78      0.81      0.79      1593
           1       0.80      0.77      0.78      1593

    accuracy                           0.79      3186
   macro avg       0.79      0.79      0.79      3186
weighted avg       0.79      0.79      0.79      3186



In [25]:
cm = tf.math.confusion_matrix(labels=y_test,predictions=y_pred)
cm

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1284,  309],
       [ 365, 1228]])>

In [34]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

model = keras.Sequential([
    keras.layers.Dense(12, input_shape=(12,), activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(3, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define the EarlyStopping callback
#early_stopping = EarlyStopping(monitor='loss', patience=5, min_delta=0.0001)

# Train the model with early stopping
model.fit(X_train, y_train, epochs=100 )#callbacks=[early_stopping]


Epoch 1/100
399/399 [==============================] - 2s 2ms/step - loss: 0.6730 - accuracy: 0.5713
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6447 - accuracy: 0.6260
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6322 - accuracy: 0.6419
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6184 - accuracy: 0.6568
Epoch 5/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6094 - accuracy: 0.6672
Epoch 6/100
399/399 [==============================] - 1s 2ms/step - loss: 0.6019 - accuracy: 0.6733
Epoch 7/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5904 - accuracy: 0.6892
Epoch 8/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5843 - accuracy: 0.6925
Epoch 9/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5758 - accuracy: 0.7012
Epoch 10/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5710 - accura

399/399 [==============================] - 1s 3ms/step - loss: 0.5140 - accuracy: 0.7557
Epoch 82/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5152 - accuracy: 0.7503
Epoch 83/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5133 - accuracy: 0.7538
Epoch 84/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5123 - accuracy: 0.7517
Epoch 85/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5141 - accuracy: 0.7533
Epoch 86/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5083 - accuracy: 0.7564
Epoch 87/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5194 - accuracy: 0.7498
Epoch 88/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5156 - accuracy: 0.7496
Epoch 89/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5109 - accuracy: 0.7513
Epoch 90/100
399/399 [==============================] - 1s 3ms/step - loss: 0.5109 - accuracy: 

In [33]:
model.evaluate(X_test,y_test)

100/100 [==============================] - 0s 2ms/step - loss: 0.4616 - accuracy: 0.7718


[0.46164628863334656, 0.7718141674995422]